In [ ]:
import streamlit as st
from datetime import date
import yfinance as yf
from prophet import Prophet
from prophet.plot import plot_plotly
from plotly import graph_objs as go
import pandas as pd

START = "2015-01-01"
TODAY = date.today().strftime("%Y-%m-%d")

st.title('Stock Forecast App')

stocks = ('GOOG', 'AAPL', 'MSFT', 'GME', 'TSLA', 'AMZN', 'NVDA')
selected_stock = st.selectbox('Select dataset for prediction', stocks)

n_years = st.slider('Years of prediction:', 1, 4)
period = n_years * 365

@st.cache_data
def load_data(ticker):
    try:
        data = yf.download(ticker, START, TODAY)
        data.reset_index(inplace=True)
        return data
    except Exception as e:
        st.error(f"Error loading data: {e}")
        return pd.DataFrame()

data_load_state = st.text('Loading data...')
data = load_data(selected_stock)

if data.empty:
    st.error("Failed to load data. Please try again.")
    st.stop()

data_load_state.text('Loading data... done!')

st.subheader('Raw data')
st.write(data.tail())

# Plot raw data
def plot_raw_data():
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=data['Date'], y=data['Open'], name="stock_open"))
    fig.add_trace(go.Scatter(x=data['Date'], y=data['Close'], name="stock_close"))
    fig.layout.update(
        title_text='Time Series data with Rangeslider', 
        xaxis_rangeslider_visible=True,
        xaxis_title="Date",
        yaxis_title="Price (USD)"
    )
    st.plotly_chart(fig, use_container_width=True)

plot_raw_data()

# Predict forecast with Prophet
df_train = data[['Date', 'Close']].copy()
df_train = df_train.rename(columns={"Date": "ds", "Close": "y"})

# Remove any NaN values
df_train = df_train.dropna()

if len(df_train) < 2:
    st.error("Not enough data points for forecasting.")
    st.stop()

with st.spinner('Training Prophet model...'):
    m = Prophet(daily_seasonality=True)
    m.fit(df_train)
    
    future = m.make_future_dataframe(periods=period)
    forecast = m.predict(future)

# Show and plot forecast
st.subheader('Forecast data')
st.write(forecast.tail())

st.write(f'Forecast plot for {n_years} years')
fig1 = plot_plotly(m, forecast)
st.plotly_chart(fig1, use_container_width=True)

st.write("Forecast components")
fig2 = m.plot_components(forecast)
st.pyplot(fig2)

# Additional metrics
st.subheader('Forecast Summary')
last_actual = df_train['y'].iloc[-1]
last_forecast = forecast['yhat'].iloc[-1]
change_percent = ((last_forecast - last_actual) / last_actual) * 100

col1, col2, col3 = st.columns(3)
with col1:
    st.metric("Current Price", f"${last_actual:.2f}")
with col2:
    st.metric("Forecasted Price", f"${last_forecast:.2f}")
with col3:
    st.metric("Predicted Change", f"{change_percent:.2f}%")

# Download forecast data
csv = forecast.to_csv(index=False)
st.download_button(
    label="Download forecast data as CSV",
    data=csv,
    file_name=f'{selected_stock}_forecast.csv',
    mime='text/csv'
)